이심축 (Angleblock의 형상을 원으로 했을 때, 그 원의 중심)을 기준으로 자석이 배치된 코드입니다.

이심축은 주 회전축 (B-CRA)에서 25mm 떨어져있습니다.

경량 모델이 아닌, Angleblock에 오로지 자석이 한 겹만 들어가 LAG 가 무거웠던 모델입니다.

## Preview : 자석 기본 Angleblock 제작 DXF

### Outer Magnet DXF
**distbtw = 0.5mm, BEST**
  
<img src="./output_1.png" width="300"/>

In [ ]:
import ezdxf
import math
import matplotlib.pyplot as plt

from ezdxf import recover
from ezdxf.addons.drawing import matplotlib
import numpy as np

from rectangle import *

# 모든 각도는 라디안 단위로 표현됩니다.
# rectangle : (중심 x, 중심 y, 너비, 높이, 회전)

# 주어진 DXF 파일을 불러와 PNG로 저장하는 함수
def display_dxf(filename):
    # 예외 처리는 코드의 간결성을 위해 생략되었습니다.
    doc, auditor = recover.readfile(str(filename) + '.dxf')
    if not auditor.has_errors:
        matplotlib.qsave(doc.modelspace(), str(filename) + '.png')

# 두 점 (point1, point2) 사이의 기울기를 계산하는 함수
def gradient(point1, point2):
    x1, y1 = point1
    x2, y2 = point2
    
    # x 값이 동일한 경우, 즉 수직선의 경우 기울기가 무한대가 되므로 이를 처리
    if x2 - x1 == 0:
        return float('inf')  # 기울기가 무한대인 경우 반환
    
    return (y2 - y1) / (x2 - x1)

# 두 각도 a와 b의 차이를 계산하는 함수
def anglediff(a, b): 
    k = abs(a - b) % 180
    if abs(k - 180) < k:
        return k - 180
    else:
        return k

In [ ]:
# 설계 정보
deltay = 0  # mm, LAG delta y
r0 = 60
r_AngleBlock = r0 + deltay

# 주축과 AngleBlock축 (편의상 이심축)의 기울기 관계
theta = 0  # AngleBlock 축에 대한 초기 각도

# 주축 기준 위치 계산
angleblock_theta = []  # 각도 (theta) 저장용 리스트
angleblock_x = []  # x 좌표 저장용 리스트
angleblock_y = []  # y 좌표 저장용 리스트
angleblock_ = []  # (theta, x, y) 튜플 저장용 리스트
angleblock_dot = []  # 점 (x, y) 저장용 리스트

In [ ]:
# 주어진 각도 범위에서 각도를 변화시키며 x, y 좌표 계산
for theta in np.arange(start=0, stop=2 * math.pi, step=2 * math.pi / 1000):
    x = math.cos(theta) * (r_AngleBlock)
    y = math.sin(theta) * (r_AngleBlock)
    angleblock_theta.append(np.arctan2(x, y))
    angleblock_x.append(x)
    angleblock_y.append(y)
    angleblock_dot.append((x, y))
    angleblock_.append((theta, x, y))

# 여기까지 AngleBlock 변환 완료
# 여기서는 이심축 (AngleBlock) 축 중심에 대한 거리를 나타낸 것입니다.
# 실 적용시에는 주회전축 (CRA)에 대해 축의 거리를 나타내고 자석의 방향을 정하는 것이 최선입니다.

In [ ]:
# 회전된 사각형을 생성하는 함수
def create_rotated_rectangle(rect):
    centerx, centery, width, height, angle = rect
    # 사각형의 각 꼭지점을 계산
    half_width = width / 2.0
    half_height = height / 2.0
    
    points = [
        (-half_width, -half_height),
        (half_width, -half_height),
        (half_width, half_height),
        (-half_width, half_height),
        (-half_width, -half_height)  # 루프를 닫기 위해 마지막 점을 다시 추가
    ]

    # 점들을 회전시키고 평행 이동
    rotated_points = []
    for x, y in points:
        # 회전
        xr = x * math.cos(angle) - y * math.sin(angle)
        yr = x * math.sin(angle) + y * math.cos(angle)
        # 평행 이동
        xt = xr + centerx
        yt = yr + centery
        rotated_points.append((xt, yt))

    # 회전된 점들로 사각형을 생성하여 모델 공간에 추가
    msp.add_lwpolyline(rotated_points)



In [ ]:
CRA_point = (0, 25)  # CRA 기준 점 설정
#만일 자석이 수차 회전 중심점 (B-CRA)가 아닌 Angleblock 중심을 바라보게 하기 위해서는 (기준 점을 (0, 0)으로 변경) CRA_point = (0, 0)

magnet_dim = (10, 3)  # 자석의 크기 설정 (너비, 높이)
magnet_margin = 0.2  # 자석 여백 설정
maghole_dim = (magnet_dim[0] + magnet_margin, magnet_dim[1] + magnet_margin)  # 자석 홀 크기 설정
distance_btw = 0.1  # 자석 간의 최소 거리 설정

In [ ]:
dot_cnt = len(angleblock_x)

mag_cnt = 1
mag_dot = []  # 자석 위치 (점) 저장용 리스트
mag_x  = []  # 자석의 x 좌표 저장용 리스트
mag_y = []  # 자석의 y 좌표 저장용 리스트
mag_deg = []  # 자석의 회전 각도 저장용 리스트
mag_rect = []  # 자석의 사각형 정보 저장용 리스트

mag_dot.append((angleblock_x[0], angleblock_y[0]))
mag_x.append(angleblock_x[0])
mag_y.append(angleblock_y[0])
mag_deg.append(0)
mag_rect.append((angleblock_x[0], angleblock_y[0], maghole_dim[0], maghole_dim[1], 0))

save_all_dots = []  # 모든 점 저장용 리스트

# 자석 간의 거리 조정 없이 첫 번째 시도
doc = ezdxf.new()

# 모델 공간을 가져옴 (모든 도형 요소가 저장되는 곳)
msp = doc.modelspace()

for i in range(1000):
    
    previous_center_x = mag_x[mag_cnt - 1] 
    previous_center_y = mag_y[mag_cnt - 1]
    previous_deg = mag_deg[mag_cnt - 1]

    center_x = angleblock_x[i]
    center_y = angleblock_y[i]
    deg = np.arctan(gradient(CRA_point, (center_x, center_y))) + 1/2 * math.pi
    r1 = (center_x, center_y, maghole_dim[0], maghole_dim[1], deg)
    r2 = (previous_center_x, previous_center_y, maghole_dim[0], maghole_dim[1], previous_deg)

    # 사각형 간의 최소 거리가 distance_btw 이상인지 확인
    if ((min_distance_between_rectangles(r1, r2) >= distance_btw) and min_distance_between_rectangles(r1, mag_rect[0]) > 0):
        create_rotated_rectangle((center_x, center_y, maghole_dim[0], maghole_dim[1], deg))
        mag_cnt += 1
        mag_dot.append((center_x, center_y))
        mag_x.append(center_x)
        mag_y.append(center_y)
        mag_deg.append(deg)
        mag_rect.append(r1)

        rect_dot = get_corners((center_x, center_y, maghole_dim[0], maghole_dim[1], deg))
        for dots in rect_dot:
            save_all_dots.append(dots)

In [ ]:
# 점을 이동시키는 함수
def move_dot(center, dot, offset):
    # 중심에서 점으로의 방향 벡터 계산
    direction = np.array(dot) - np.array(center)
    
    # 방향 벡터를 정규화
    norm = np.linalg.norm(direction)
    if norm == 0:
        return dot  # 점이 중심에 있는 경우 점을 그대로 반환
    direction /= norm
    
    # 오프셋 적용 후 새로운 위치 계산
    new_dot = np.array(dot) + offset * direction
    return new_dot.tolist()

# DXF 파일에서 점을 오프셋시키는 함수
def offset_dxf(center, dots, offset_p, offset_n):

    dots_p = []
    dots_n = []
    dots_p.append(move_dot(center, dots[0], offset_p))
    dots_n.append(move_dot(center, dots[0], offset_n))

    for i in range(1, len(dots) + 1):
        dots_p.append(move_dot(center, dots[i % len(dots)], offset_p))
        dots_n.append(move_dot(center, dots[i % len(dots)], offset_n))

        # 중심 점과 오프셋된 점 사이에 선을 그립니다.
        msp.add_line(dots_p[i - 1], dots_p[i % len(dots)], dxfattribs={'color': 3})  # 초록색 선
        msp.add_line(dots_n[i - 1], dots_n[i % len(dots)], dxfattribs={'color': 5})  # 파란색 선


In [ ]:
#자석을 넣는 홈을 감싸기 위한 최대/최소 거리 구하기

maxd = 0
mind = 1000000

#(0, 0)은 Angleblock의 중심으로, 수차 회전축과 25mm 떨어져있습니다.
for dots in save_all_dots:
    maxd = max(maxd, distance(dots, (0, 0)))
    mind = min(mind, distance(dots, (0, 0)))
offset_dxf((0, 0), angleblock_dot, (maxd - r_AngleBlock + distance_btw), -(r_AngleBlock - mind - distance_btw))

print("maxd", (maxd + distance_btw))
# DXF 문서를 파일로 저장
doc.saveas('output_1.dxf')

<img src="./output_1.png" alt="설명 텍스트" width="300"/>

- 직사각형 틀로 여러 개의 네오디뮴 자석을 고정
- 외곽 (초록색 선) 내부 (파란색 선) 으로 둘러 고정 가능하도록 설계

In [ ]:
# 주어진 거리로 크기 조정
# 초깃 값으로 우선 만든 이후, 위 그림의 오른쪽에 보이는 간극을 최대한 좁히기 위해 자석간의 최소 간격 변수인 min_distance를 조금씩 높이며 최적화를 하기 위한 코드


dd_ = min_distance_between_rectangles(mag_rect[-1], mag_rect[0])

distance_btw = ((distance_btw * mag_cnt) + dd_) / mag_cnt

print("distance btw =", distance_btw)

# 새로운 DXF 문서 생성
doc = ezdxf.new()

# 모델 공간을 가져옴 (모든 도형 요소가 저장되는 곳)
msp = doc.modelspace()

mag_cnt = 1
mag_dot = []
mag_x  = []
mag_y = []
mag_deg = []  # 절대좌표 기준 회전. 
mag_rect = []

mag_dot.append((angleblock_x[0], angleblock_y[0]))
mag_x.append(angleblock_x[0])
mag_y.append(angleblock_y[0])
mag_deg.append(0)
mag_rect.append((angleblock_x[0], angleblock_y[0], maghole_dim[0], maghole_dim[1], 0))

save_all_dots = []

for i in range(1000):
    
    previous_center_x = mag_x[mag_cnt - 1] 
    previous_center_y = mag_y[mag_cnt - 1]
    previous_deg = mag_deg[mag_cnt - 1]

    center_x = angleblock_x[i]
    center_y = angleblock_y[i]
    deg = np.arctan(gradient(CRA_point, (center_x, center_y))) + 1/2 * math.pi
    r1 = (center_x, center_y, maghole_dim[0], maghole_dim[1], deg)
    r2 = (previous_center_x, previous_center_y, maghole_dim[0], maghole_dim[1], previous_deg)

    if ((min_distance_between_rectangles(r1, r2) >= distance_btw) and min_distance_between_rectangles(r1, mag_rect[0]) > 0):
        create_rotated_rectangle((center_x, center_y, maghole_dim[0], maghole_dim[1], deg))
        mag_cnt += 1
        mag_dot.append((center_x, center_y))
        mag_x.append(center_x)
        mag_y.append(center_y)
        mag_deg.append(deg)
        mag_rect.append(r1)

        rect_dot = get_corners((center_x, center_y, maghole_dim[0], maghole_dim[1], deg))
        for dots in rect_dot:
            save_all_dots.append(dots)

maxd = 0
mind = 1000000
for dots in save_all_dots:
    maxd = max(maxd, distance(dots, (0, 0)))
    mind = min(mind, distance(dots, (0, 0)))

# DXF 문서를 파일로 저장
doc.saveas('output_2.dxf')

print(maxd, mind)
print(maxd - r_AngleBlock, mind - r_AngleBlock)
print(maxd - mind)

# 그래프 그리기
plt.scatter(angleblock_x, angleblock_y)
plt.scatter(CRA_point[0], CRA_point[1])
ax = plt.gca()
ax.set_aspect('equal', adjustable='box')

plt.show()

# DXF 파일을 이미지로 변환하여 표시
display_dxf('output_1')
display_dxf('output_2')